### Extract Discovery set from Danenberg_et_al
* 1) data filtering: exclude patients with no clinical data and images with less than 500 cells
* 2) Random partition: for reproducibility purpose, we use a random seed.

### Standardize data, and Generate cellular graphs

### Identify TME patterns with Soft WL subtree kernel